# Time Series Forecasting with LSTM (PyTorch)

Long Short-Term Memory (LSTM) networks are a type of Recurrent Neural Network (RNN) capable of learning order dependence in sequence prediction problems.

**Objective:** Predict the next value in a sine wave (proxy for a cyclical asset pattern).

**Steps:**
1.  Generate synthetic data.
2.  Preprocess sequences (Sliding Window).
3.  Build LSTM model in PyTorch.
4.  Train and Evaluate.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 1. Generate Data
t = np.linspace(0, 100, 1000)
data = np.sin(t)

# 2. Sliding Window (Lookback)
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 20
X, y = create_sequences(data, seq_length)

# Convert to Tensor
X_train = torch.from_numpy(X).float().unsqueeze(2) # (Batch, Seq, Feature)
y_train = torch.from_numpy(y).float()

# 3. LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

model = LSTMModel()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 4. Train Loop
epochs = 10
print("Training...")
for i in range(epochs):
    for seq, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        y_pred = model(seq)
        single_loss = loss_function(y_pred, labels.unsqueeze(0))
        single_loss.backward()
        optimizer.step()
    if i % 2 == 0:
        print(f'Epoch: {i} Loss: {single_loss.item():.5f}')

print("Training Complete.")

# 5. Prediction (Validation)
model.eval()
test_seq = X_train[0]
with torch.no_grad():
    pred = model(test_seq)
    print(f"Target: {y_train[0].item():.4f}, Predicted: {pred.item():.4f}")